## Spark Notebook Setup

In [33]:
import $ivy.`org.apache.spark::spark-sql:2.4.0`
import org.apache.spark.sql._
import org.apache.spark.{SparkConf, SparkContext}

import $ivy.$
import org.apache.spark.sql._
import org.apache.spark.{SparkConf, SparkContext}

In [34]:
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.log4j.{Level, Logger}

In [35]:
val spark = {
  NotebookSparkSession.builder()
    .master("local[*]")
    .getOrCreate()
}

Getting spark JARs

Creating SparkSession

Spark UI

spark: SparkSession = org.apache.spark.sql.SparkSession@39ff85d5

## Preprocessing Data

In [36]:
val moviePath = "notebooks/data/movies.csv"
val stopWordsPath = "notebooks/data/stopwords.txt"
val lemmatizationPath = "notebooks/data/lemmatization.txt"

moviePath: String = "notebooks/data/movies.csv"
stopWordsPath: String = "notebooks/data/stopwords.txt"
lemmatizationPath: String = "notebooks/data/lemmatization.txt"

In [37]:
val moviesRaw = spark.read
    .option("header", "true")
    .option("multiline", "true")
    .option("escape", "\"")
    .csv(moviePath)
    .rdd
    .zipWithIndex
    .map(x => (x._2, x._1.getString(0), x._1.getString(1), x._1.getString(7))) // (id, title, genres)

moviesRaw.take(5).foreach(println)

csv at cmd37.sc:5

csv at cmd37.sc:5

take at cmd37.sc:10

take at cmd37.sc:10

(0,1901,Kansas Saloon Smashers,A bartender is working at a saloon, serving drinks to customers. After he fills a stereotypically Irish man's bucket with beer, Carrie Nation and her followers burst inside. They assault the Irish man, pulling his hat over his eyes and then dumping the beer over his head. The group then begin wrecking the bar, smashing the fixtures, mirrors, and breaking the cash register. The bartender then sprays seltzer water in Nation's face before a group of policemen appear and order everybody to leave.[1])
(1,1901,Love by the Light of the Moon,The moon, painted with a smiling face hangs over a park at night. A young couple walking past a fence learn on a railing and look up. The moon smiles. They embrace, and the moon's smile gets bigger. They then sit down on a bench by a tree. The moon's view is blocked, causing him to frown. In the last scene, the man fans the woman with his hat because the moon has left the sky and is perched over her shoulder to see everything

moviesRaw: RDD[(Long, String, String, String)] = MapPartitionsRDD[105] at map at cmd37.sc:8

In [38]:
val stopWords = spark.read
    .textFile(stopWordsPath)
    .rdd
    .map(_.trim.toLowerCase.replaceAll("[^a-z]", ""))
    .collect()
    .toSet

collect at cmd38.sc:4

collect at cmd38.sc:4

stopWords: Set[String] = Set(
  "quotient",
  "snow",
  "liquid",
  "eye",
  "down",
  "wash",
  "rub",
  "side",
  "measure",
  "please",
  "trouble",
  "finger",
  "read",
  "number",
  "mass",
  "able",
  "behind",
  "mother",
  "chief",
  "hurry",
  "for",
  "drive",
  "magnet",
  "tire",
  "find",
  "neighbor",
  "wild",
  "support",
  "produce",
  "school",
  "rock",
  "soldier",
  "weight",
  "question",
  "art",
  "parent",
  "shop",
  "method",
...

In [39]:
val lemmatization = spark.read
    .textFile(lemmatizationPath)
    .rdd
    .map(_.split("\\s+"))
    .map(x => (x(1), x(0)))
    .collect()
    .toMap

lemmatization.take(5).foreach(println)

collect at cmd39.sc:5

collect at cmd39.sc:5

(professed,profess)
(pathogens,pathogen)
(purifies,purify)
(phosphates,phosphate)
(buns,bun)


lemmatization: Map[String, String] = Map(
  "professed" -> "profess",
  "pathogens" -> "pathogen",
  "purifies" -> "purify",
  "phosphates" -> "phosphate",
  "buns" -> "bun",
  "fathering" -> "father",
  "soapiest" -> "soapy",
  "basils" -> "basil",
  "showdowns" -> "showdown",
  "redcurrants" -> "redcurrant",
  "night-watchmen" -> "night-watchman",
  "regularizing" -> "regularize",
  "boutiques" -> "boutique",
  "satsumas" -> "satsuma",
  "healings" -> "healing",
  "breaks" -> "break",
  "cut-backs" -> "cut-back",
  "sneezed" -> "sneeze",
  "forgotten" -> "forget",
  "insinuates" -> "insinuate",
  "stocktakings" -> "stocktaking",
  "cartwheeled" -> "cartwheel",
  "boxers" -> "boxer",
  "dynasties" -> "dynasty",
  "hydrologists" -> "hydrologist",
  "sectors" -> "sector",
  "radiopharmaceuticals" -> "radiopharmaceutical",
  "embedded" -> "embed",
  "rationalising" -> "rationalise",
  "respecting" -> "respect",
  "exorcists" -> "exorcist",
  "allotments" -> "allotment",
  "styrenes" -> "

In [40]:
val movies = moviesRaw.map { case (id, year, title, plot) =>
    val processedPlot = plot.toLowerCase
        .replaceAll("[^a-z ]", " ")
        .split(" ")
        .filterNot(stopWords.contains)
        .filterNot(_.isEmpty)
        .map(token => lemmatization.getOrElse(token, token))

    (id, year, title, processedPlot)
}

movies.take(5).foreach(println)

take at cmd40.sc:12

take at cmd40.sc:12

(0,1901,Kansas Saloon Smashers,[Ljava.lang.String;@60ecfa8e)
(1,1901,Love by the Light of the Moon,[Ljava.lang.String;@7e792983)
(2,1901,The Martyred Presidents,[Ljava.lang.String;@51e4c3dd)
(3,1901,Terrible Teddy, the Grizzly King,[Ljava.lang.String;@5dbb21cd)
(4,1902,Jack and the Beanstalk,[Ljava.lang.String;@2ef9067b)


movies: RDD[(Long, String, String, Array[String])] = MapPartitionsRDD[117] at map at cmd40.sc:1

In [41]:
val vocabulary = movies 
    .flatMap { case (_, _, _, cleanedPlot) => cleanedPlot }
    .distinct

vocabulary.take(10).foreach(println)

distinct at cmd41.sc:2

take at cmd41.sc:5

mistretta
shh
sammee
bone
gerven
nothin
rostom
mislabel
fred
bresac


vocabulary: RDD[String] = MapPartitionsRDD[121] at distinct at cmd41.sc:2

In [42]:
val vocabularyIndexed = vocabulary
    .zipWithIndex
    .map(_.swap)

vocabularyIndexed.take(10).foreach(println)

take at cmd42.sc:5

take at cmd42.sc:5

(0,mistretta)
(1,shh)
(2,sammee)
(3,bone)
(4,gerven)
(5,nothin)
(6,rostom)
(7,mislabel)
(8,fred)
(9,bresac)


vocabularyIndexed: RDD[(Long, String)] = MapPartitionsRDD[123] at map at cmd42.sc:3

In [43]:
def dfToIdf(df: Int, totalDocs: Long): Double = {
    math.log(totalDocs.toDouble / df)
}

defined function dfToIdf

In [44]:
val numDocs = movies.count
val vocabularySize = vocabularyIndexed.count.toInt

count at cmd44.sc:1

count at cmd44.sc:1

count at cmd44.sc:2

count at cmd44.sc:2

numDocs: Long = 34886L
vocabularySize: Int = 112805

In [45]:
val vocabIdfs = movies
    .flatMap { case (_, _, _, cleanedPlot) => cleanedPlot }
    .map { word => (word, 1) }
    .reduceByKey(_ + _)
    .zipWithIndex
    .map { case ((word, count), id) => (word, id, count) }
    .map { case (word, id, count) => (word, id, dfToIdf(count, numDocs)) }

vocabIdfs.take(10).foreach(println)

map at cmd45.sc:3

map at cmd45.sc:3

take at cmd45.sc:9

(mistretta,0,8.513930732522766)
(shh,1,10.45984088157808)
(sammee,2,10.45984088157808)
(bone,3,5.250354728736659)
(gerven,4,10.45984088157808)
(nothin,5,9.36122859290997)
(rostom,6,8.061945608779709)
(mislabel,7,9.36122859290997)
(fred,8,3.370597726550566)
(bresac,9,10.45984088157808)


vocabIdfs: RDD[(String, Long, Double)] = MapPartitionsRDD[129] at map at cmd45.sc:7

In [46]:
import org.apache.spark.rdd.RDD
import $ivy.`org.apache.spark::spark-mllib:2.4.0`
import org.apache.spark.mllib.linalg.{Vector, Vectors}

import org.apache.spark.rdd.RDD
import $ivy.$
import org.apache.spark.mllib.linalg.{Vector, Vectors}

In [47]:
def tokenListToTfIdf(tokenList: List[String], vocabulary: RDD[(String, Long, Double)]): List[Double] = {
    val tfs = spark.sparkContext.parallelize(tokenList)
        .map((_, 1))
        .reduceByKey(_ + _)

    val idfs = vocabulary 
        .map { case (word, id, idf) => (word, (id, idf)) }

    val tfidfs = tfs.rightOuterJoin(idfs)
        .map { case (word, (tfOpt, (id, idf))) => (word, tfOpt.getOrElse(0), idf, id) }
        .map { case (word, tf, idf, id) => (id, tf * idf) }
        .sortBy(_._1)
        .collect
        .map(_._2)
        .toList

    tfidfs
}

defined function tokenListToTfIdf

In [47]:
// val tokenList = List("apple", "banana", "orange", "apple")
// val vocabulary = spark.sparkContext.parallelize(Seq(("apple", 0, 2), ("banana", 1, 2), ("orange", 2, 1)))

In [47]:
// val vocabIdfs = vocabulary
//     .map { case (word, id, df) => (word, id, dfToIdf(df, 3)) }
// 
// val tf = tokenListToTfIdf(tokenList, vocabIdfs)
// println("test", tf)

In [48]:
val firstEntry = movies.first()

first at cmd48.sc:1

first at cmd48.sc:1

firstEntry: (Long, String, String, Array[String]) = (
  0L,
  "1901",
  "Kansas Saloon Smashers",
  Array(
    "bartender",
    "work",
    "saloon",
    "serve",
    "drink",
    "customer",
    "fill",
    "stereotypically",
    "irish",
    "s",
    "bucket",
    "beer",
    "carrie",
    "follower",
    "burst",
    "inside",
    "assault",
    "irish",
    "pull",
    "eye",
    "dump",
    "beer",
    "wreck",
    "smash",
    "fixture",
    "mirror",
    "break",
    "cash",
    "register",
    "bartender",
    "spray",
    "seltzer",
    "s",
    "policeman",
...

In [49]:
vocabIdfs.take(10).foreach(println)

take at cmd49.sc:1

take at cmd49.sc:1

(mistretta,0,8.513930732522766)
(shh,1,10.45984088157808)
(sammee,2,10.45984088157808)
(bone,3,5.250354728736659)
(gerven,4,10.45984088157808)
(nothin,5,9.36122859290997)
(rostom,6,8.061945608779709)
(mislabel,7,9.36122859290997)
(fred,8,3.370597726550566)
(bresac,9,10.45984088157808)


In [50]:
val firstEntryTfIdf = tokenListToTfIdf(firstEntry._4.toList, vocabIdfs)
println(firstEntryTfIdf)

map at cmd47.sc:3

map at cmd47.sc:3

map at cmd47.sc:7

map at cmd47.sc:7

sortBy at cmd47.sc:12

...

sortBy at cmd47.sc:12

collect at cmd47.sc:12

collect at cmd47.sc:12

List(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

firstEntryTfIdf: List[Double] = List(
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
...

In [51]:
println(firstEntryTfIdf.sum)

136.8393980256314


In [52]:
def countTokens(tokens: List[String]): Map[String, Int] = {
    tokens.groupBy(identity).mapValues(_.size)
}

defined function countTokens

In [53]:
val tokenCounts = movies
    .flatMap { case (id, title, genres, plotTokens) => {
        val tfs = plotTokens
            .groupBy(identity)
            .mapValues(_.size)
            .map { case (token, count) => (token, id, count) }
            .toList
        tfs
    }}

tokenCounts.take(10).foreach(println)

take at cmd53.sc:11

take at cmd53.sc:11

(eye,0,1)
(register,0,1)
(s,0,2)
(wreck,0,1)
(break,0,1)
(serve,0,1)
(burst,0,1)
(everybody,0,1)
(carrie,0,1)
(dump,0,1)


tokenCounts: RDD[(String, Long, Int)] = MapPartitionsRDD[144] at flatMap at cmd53.sc:2

In [54]:
vocabIdfs.take(10).foreach(println)

take at cmd54.sc:1

take at cmd54.sc:1

(mistretta,0,8.513930732522766)
(shh,1,10.45984088157808)
(sammee,2,10.45984088157808)
(bone,3,5.250354728736659)
(gerven,4,10.45984088157808)
(nothin,5,9.36122859290997)
(rostom,6,8.061945608779709)
(mislabel,7,9.36122859290997)
(fred,8,3.370597726550566)
(bresac,9,10.45984088157808)


In [64]:
import scala.collection.immutable.Map

import scala.collection.immutable.Map

In [65]:
def calc_magnitude(v: List[Double]) = {
  math.sqrt(v.map(x => x * x).sum)
}

def calc_cosign(doc_map_1: Map[Long, Double], doc_map_2: Map[Long, Double]) = {
  //This is a wierd way of doing the dot product, but it works.
  //So, for every value in map_1, we get the corresponding value in map_2
  //If the value isn't in map_2, include it as a 0
  //If the value is in map_2 and not map_1, then don't include it
  //Not including a value be the same as putting a zero
  //then, simply sum the list
  val dot_product = doc_map_1
    .map({case (k, v) => (k, (v, doc_map_2.get(k)))})
    .map({
      case (k, (v1, Some(v2))) => (v1 * v2)
      case (k, (v1, None)) => (0)
    }).sum

  val v1 = doc_map_1.map({case (k, v) => v}).toList
  val v2 = doc_map_2.map({case (k, v) => v}).toList

  dot_product / (calc_magnitude(v1) * calc_magnitude(v2))
}


defined function calc_magnitude
defined function calc_cosign

In [62]:
val tfIdfs = tokenCounts
    .map { case (token, docId, tf) => (token, (docId, tf)) }
    .join(vocabIdfs.map { case (word, wordId, idf) => (word, (wordId, idf)) })
    .map { case (token, ((docId, tf), (wordId, idf))) => (docId, (wordId, tf * idf)) }
    .groupByKey
    .mapValues(_.toMap)

tfIdfs.take(10).foreach(println)

map at cmd62.sc:2

map at cmd62.sc:2

map at cmd62.sc:3

map at cmd62.sc:3

map at cmd62.sc:4

map at cmd62.sc:4

take at cmd62.sc:8

take at cmd62.sc:8

(19021,Map(42181 -> 4.038218613771562, 84168 -> 1.878359200279481, 110851 -> 40.78627894292018, 92548 -> 6.103132054888489, 108663 -> 4.051312090518582, 89888 -> 2.3993008350394405, 13110 -> 3.0653477743590423, 105116 -> 0.6733928911433047, 84915 -> 3.328142371111169, 4739 -> 4.37990768648249, 54027 -> 3.5641581838302123, 32823 -> 4.286054777676143, 75016 -> 3.3107092830206737, 25639 -> 1.4977054915114785, 68266 -> 2.2426722858120103, 44880 -> 5.71490875321483, 99973 -> 2.1842098270000703, 11913 -> 2.7465029927062097, 77305 -> 5.107982748102014, 23204 -> 3.8371045576282405, 109115 -> 0.7377553087765231, 10982 -> 9.552522228478797, 12924 -> 3.4131936037293245, 84393 -> 3.2828221156681803, 32484 -> 2.1345348518254985, 15505 -> 4.448573707173918, 24006 -> 5.329942166655006, 79331 -> 3.1972122806038388, 55608 -> 3.123555221556783, 56794 -> 2.138176074443079, 14291 -> 4.119481577850328, 1836 -> 5.161098792991619, 89564 -> 4.793414193465648, 111124 -> 2.5981138037540994, 650 -> 7.11936888382

tfIdfs: RDD[(Long, Map[Long, Double])] = MapPartitionsRDD[178] at mapValues at cmd62.sc:6

In [66]:
def cosineBetweenDocs(doc1: Int, doc2: Int, tfIdfs: RDD[(Long, Map[Long, Double])]): Double = {
    val doc1TfIdfs = tfIdfs.filter(_._1 == doc1).collect.head._2
    val doc2TfIdfs = tfIdfs.filter(_._1 == doc2).collect.head._2

    calc_cosign(doc1TfIdfs, doc2TfIdfs)
}

defined function cosineBetweenDocs

In [67]:
cosineBetweenDocs(0, 1, tfIdfs)

collect at cmd66.sc:2

collect at cmd66.sc:2

collect at cmd66.sc:3

collect at cmd66.sc:3

res67: Double = 0.01807862983103303

In [68]:
cosineBetweenDocs(4, 4, tfIdfs)

collect at cmd66.sc:2

collect at cmd66.sc:2